# Ce notebook vise à extraire des informations pertinentes à partir de journaux d'oncologie en utilisant le modèle Gemini Pro 1.5.

## Installation des bibliothèques nécessaires

In [ ]:
!pip -q install langchain_experimental langchain_core
!pip -q install google-generativeai==0.3.1
!pip -q install google-ai-generativelanguage==0.4.0
!pip -q install langchain-google-genai
!pip -q install "langchain[docarray]"
!pip show langchain langchain-core
!pip install pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.6/177.6 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 271.6/271.6 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 810.5/810.5 kB 39.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 56.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.6/146.6 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.4/137.4 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.3/215.3 kB 4.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing meta

In [ ]:
#@title Configuration de l'authentification
import os
import google.generativeai as genai
from google.colab import userdata
import json
os.environ["GOOGLE_API_KEY"] = userdata.get('gemini')
genai.configure(api_key=os.environ["GOOGLE_API_KEY"])

[Obtenez votre clé ici](https://aistudio.google.com/app/prompts/new_chat?utm_source=agd&utm_medium=referral&utm_campaign=core-cta&utm_content=)

Pour lister tous les modèles disponibles :

In [ ]:
models = [m for m in genai.list_models()]
models

[Model(name='models/chat-bison-001',
       base_model_id='',
       version='001',
       display_name='PaLM 2 Chat (Legacy)',
       description='A legacy text-only model optimized for chat conversations',
       input_token_limit=4096,
       output_token_limit=1024,
       supported_generation_methods=['generateMessage', 'countMessageTokens'],
       temperature=0.25,
       top_p=0.95,
       top_k=40),
 Model(name='models/text-bison-001',
       base_model_id='',
       version='001',
       display_name='PaLM 2 (Legacy)',
       description='A legacy model that understands text and generates text as an output',
       input_token_limit=8196,
       output_token_limit=1024,
       supported_generation_methods=['generateText', 'countTextTokens', 'createTunedTextModel'],
       temperature=0.7,
       top_p=0.95,
       top_k=40),
 Model(name='models/embedding-gecko-001',
       base_model_id='',
       version='001',
       display_name='Embedding Gecko',
       description='Obtai

# Nous avons testé le modèle avec l'observation3


In [ ]:
import os
import json
from langchain.chains.question_answering import load_qa_chain
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate
from langchain.document_loaders import PyPDFLoader



#Pour la génération du prompt, nous avons utilisé la bibliothèque LangChain
prompt_template = """

  You are a medical expert assistant. Answer the question as detailed as possible  from the provided context, make sure to provide all the details, if the answer is not in
  provided context just write 'Not specified', and give me this information in a json format ,don't provide the wrong answer\n\n
  Context:\n {context}?\n
  Question: \n{question}\n

  Answer:

"""

prompt = PromptTemplate(template=prompt_template, input_variables=["context", "question"])
model = ChatGoogleGenerativeAI(model="gemini-pro", temperature=0.3)
chain = load_qa_chain(model, chain_type="stuff", prompt=prompt)

folder_path = "/content/pdfs"

question = "Please provide the following information in a JSON format with the specified keys:\n\
        - Patient:\n\
          - Âge\n\
          - Sexe\n\
          - Lieu\n\
          - Profession\n\
          - Téléphone\n\
        - Antécédents_médicaux:\n\
          - Personnels:\n\
            - Médicaux\n\
            - Chirurgicaux\n\
          - Familiaux:\n\
            - Cas_similaires\n\
          - Statut_hormonal:\n\
            - Ménopausée\n\
            - Années_ménopause\n\
        - Motifs_de_consultation:\n\
          - Durée\n\
          - Symptômes\n\
          - Symptômes_associés\n\
          - Contexte\n\
        - Examen_initial:\n\
          - Examen_gynécologique:\n\
            - Résultats\n\
        - Examens_paracliniques:\n\
          - Type\n\
          - Date\n\
          - Lieu\n\
          - Résultat (or Résultats if multiple results are present)\n\
        - Orientation\n\
        - Symptômes_présentés\n\
        - Examen_physique:\n\
          - Général:\n\
            - OMS\n\
            - Peau\n\
            - Conjonctive\n\
          - Abdomen\n\
          - Examen_pelvien:\n\
            - Speculum vaginal\n\
            - Toucher rectal\n\
          - Ganglions_Lymphatiques_IngUinaux\n\
        - Résumé:\n\
          - Description_patient\n\
          - Évaluation_clinique_et_traitement"



patient_data_list = []

for filename in os.listdir(folder_path):
    if filename.endswith(".pdf"):
        file_path = os.path.join(folder_path, filename)
        loader = PyPDFLoader(file_path)
        clinical_context = loader.load()
        print(clinical_context)
        response = chain({"input_documents": clinical_context, "question": question}, return_only_outputs=True)

        # pour Supprimez les triples backticks et les caractères de nouvelle ligne
        cleaned_json_str = response['output_text'].strip('```json\n').strip()

        json_dict = json.loads(cleaned_json_str)

        # Pour Ajouter le nom du PDF au dictionnaire
        json_dict["file_name"] = filename

        patient_data_list.append(json_dict)

        print(f"Medical information from {filename} added to the list")
        print()


with open('patient_data.json', 'w') as json_file:
    json.dump(patient_data_list, json_file, indent=4, ensure_ascii=False)

print("Toutes les informations médicales ont été enregistrées dans : patient_data.json")


[Document(page_content="Nom et Prénom :\nAge: Genre: Etat. Civil: F A AUTRE  44IP:\nLit:Identification du patient :*1\xa0287\xa0485*\nObservations :JOURNAL DES OBSERVATIONS \nCLINIQUES\n    Service:SERVICE DE \n10/02/2020  00:00:00 Date: YESSOUFOU MOHAMED médecin\nEXAMEN PROCTOLOGIQUE DU 06/02/2020 normale\nInspection/\nABSENCEDEMARIQUE HEMORROIDAIRES\nPAS DE FISSURE NI DE FISTULE\nPAS DE PROLAPSUS HEMORROIDAIRE A L'EFFORT NI AU REPOS\nTOUCHER RECTAL\nNON DOULOUREUX\nBONNE TONICITE SPHYNCTERIENNE\nPAS DE MASSE \nDOIGTIER REVIENT PROPRE AU RETRAIT\nAnuscopie/Rectoscopie: progression jusqu'au 15 cm de la MA ; muqueuse macroscopiquement normale\n++>Simulation ce jourSERVICE DE RADIOTHÉRAPIE service:\n05/02/2020  00:00:00 Date: YESSOUFOU MOHAMED médecin\nAvant de simuler la patiente\n=>>Expliquer a la patiente le déroulement de la rth\n=>L'existance d'éventuels effet secondaires\n=>La necessité de ne pas interrompre sa rth\n=>l'existance d'une surveillance hebdomadaire\nVu qu'il y a une at

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


Medical information from observation3.pdf added to the list

Toutes les informations médicales ont été enregistrées dans : patient_data.json


# Afficher les résultats

In [ ]:
import json

with open('patient_data.json', 'r') as json_file:
    patient_data_list = json.load(json_file)

print(json.dumps(patient_data_list, indent=4, ensure_ascii=False))


[
    {
        "Patient": {
            "Âge": "44",
            "Sexe": "F",
            "Lieu": "Chefchaoun",
            "Profession": "Non spécifié",
            "Téléphone": "0700370019"
        },
        "Antécédents_médicaux": {
            "Personnels": {
                "Médicaux": "Non spécifiés",
                "Chirurgicaux": "Non spécifiés"
            },
            "Familiaux": {
                "Cas_similaires": "Non spécifiés"
            },
            "Statut_hormonal": {
                "Ménopausée": "Oui",
                "Années_ménopause": "5"
            }
        },
        "Motifs_de_consultation": {
            "Durée": "Environ 03 mois",
            "Symptômes": "Métro-hémorragies de moyenne abondance",
            "Symptômes_associés": "Aucun",
            "Contexte": "CSE"
        },
        "Examen_initial": {
            "Examen_gynécologique": {
                "Résultats": "Gros col tumoral de 05 cm avec atteintes des paramètres"
            }
     